In [27]:
# CMSC426 Project 3
# GROUP MEMBERS: Andrew Sumner (asumner1), Teimuraz Trapaidze (ttrapaid)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
from scipy import ndimage
import math
import random
import sklearn as sk
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

# CONVERT IMAGES TO GRAYSCALE?????

In [28]:
folders = ["images/dolphin", "images/airplanes", "images/Leopards"]

def get_imgs_from_folders(folders):
    image_sets = []
    for folder in folders:
        images = []
        i = 0
        for filename in os.listdir(folder):
            i = i + 1
            image = img.imread(os.path.join(folder,filename))
            images.append(image)
            if i >= 65:
                break
        image_sets.append(images)
    return image_sets

all_images = get_imgs_from_folders(folders)

In [29]:
desc_all_imgs = []
kp_all_imgs = []
all_imgs_full_data = []
min_num_kp = 999999999999999

# FIRST LOOP FINDS MIN KP BOUND
for img_set in all_images:
    for image in img_set:
        sift = cv2.SIFT_create()
        kp, descriptors = sift.detectAndCompute(image, None)
        if len(kp) < min_num_kp:
            min_num_kp = len(kp)

print("MIN NUM KP =", min_num_kp)



# SECOND LOOP FINDS BEST KEYPOINTS WHERE ALL IMGS HAVE SAME (MIN) NUM
i = -1
for img_set in all_images:
    i += 1
    img_set_full_data = []
    for image in img_set:
        sift = cv2.SIFT_create(nfeatures=min_num_kp)
        kp, descriptors = sift.detectAndCompute(image, None)
        kp = kp[:min_num_kp]
        descriptors = descriptors[:min_num_kp]
        img_set_full_data.append((image, i, kp, descriptors))
        desc_all_imgs.append(descriptors)
        kp_all_imgs = kp_all_imgs + kp
    all_imgs_full_data.append(img_set_full_data)
#     print(descriptors.shape)
#     for pt, desc in zip(kp, descriptors):
#         (x,y) = pt.pt
#         print((x,y), desc)

desc_all_imgs = tuple(desc_all_imgs)
desc_all_imgs = np.vstack(desc_all_imgs)
#print(desc_all_imgs)
print()
print(desc_all_imgs.shape)

MIN NUM KP = 34

(6630, 128)


In [41]:
# splitting train and test
# note that we already know the labels since the folders are still
# separated within the output data structure (for now)
train_x = []
test_x = []
train_y = []
test_y = []

i = -1

for img_set in all_imgs_full_data:
    i += 1
    train_x_subset, test_x_subset = train_test_split(img_set, test_size=0.1)
    train_x = train_x + train_x_subset
    test_x = test_x + test_x_subset
    train_y = train_y + (np.full(len(train_x_subset), i)).tolist()
    test_y = test_y + (np.full(len(test_x_subset), i)).tolist()
    
print(test_y)

desc_all_train_imgs = [d for (_,_,_,d) in train_x]
desc_all_train_imgs = tuple(desc_all_train_imgs)
desc_all_train_imgs = np.vstack(desc_all_train_imgs)
print(len(desc_all_train_imgs))

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]
5916


In [31]:
def clustering(keypoints, descriptors, k):
    num_descs = len(keypoints)
    
    center_descs = []
    # initial random k centers
    for i in range(k):
        center = random.randint(0, num_descs-1)
        center_descs.append(descriptors[center])
        
    check = False
    
    while (not check):
        clusters = {i:[] for i in range(k)}
            
        for descriptor in descriptors:
            closest = 99999999999
            i = 0
            for center_desc in center_descs:
                dist = np.sum((center_desc-descriptor)**2)

#                 dist = np.linalg.norm(center_desc - descriptor)
                if dist < closest:
                    closest = dist
                    closest_center = i
                i += 1

            clusters[closest_center].append(descriptor)
            
        old_centers = center_descs
        center_descs = []
        
        for center_num, cluster in clusters.items():
            if (len(cluster) > 0):
                cluster = np.vstack(cluster)
#                 new_center = np.sum(cluster, axis=0)/np.linalg.norm(cluster)
                new_center = cluster.mean(axis=0)
                center_descs.append(new_center)
        
        check = True
        for old, new in zip(old_centers, center_descs):
            if not np.array_equal(old, new):
                check = False
                break
        
        

    return center_descs

In [32]:
def clustering2(descriptors, k):
    indices = range(0, len(descriptors))
    centroid_indices = random.sample(indices, k)
    
    centroids = []
    for i in centroid_indices:
        centroids.append(descriptors[i])
    old_centroids = []
    
    flag = False
    
    while (not flag):
        centroid_groupings = {centroid_idx:[] for centroid_idx in range(len(centroids))}
        for row in descriptors:
            min_dist = 9999999999999999
            min_centroid = None
            min_centroid_idx = -1
            i = -1
            for centroid in centroids:
                i = i+1
                dist = np.linalg.norm(row - centroid)
                if dist < min_dist:
                    min_dist = dist
                    min_centroid = centroid
                    min_centroid_idx = i
            (centroid_groupings[min_centroid_idx]).append(row)
        
        old_centroids = centroids.copy()
        for i in range(len(centroids)):
            cluster_descs = centroid_groupings[i]
            #print(len(cluster_descs))
            cluster_descs = tuple(cluster_descs)
            cluster_descs = np.vstack(cluster_descs)
            new_centroid = cluster_descs.mean(axis=0)
            #print(new_centroid)
            #print(new_centroid.shape)
            centroids[i] = new_centroid
            #print(centroids)
        #print(centroids)
        #print(old_centroids)
        flag = np.array_equal(
            np.vstack(tuple(centroids)), 
            np.vstack(tuple(old_centroids))
        )
        print(flag)
        print(len(centroids))
    return centroids

In [42]:
#centers = clustering(kp_all_imgs, desc_all_imgs, 400)

centers = clustering2(desc_all_train_imgs, 400)

False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
False
400
True
400


In [43]:
print(centers)

[array([ 19.        ,   6.142857  ,  12.142858  ,  16.428572  ,
        59.142857  ,   7.        ,   3.142857  ,   6.714286  ,
       121.28571   ,  22.428572  ,  18.714285  ,   2.7142856 ,
         4.571429  ,   1.5714285 ,   2.142857  ,  42.714287  ,
        92.14286   ,  17.        ,  13.714286  ,   3.5714285 ,
         3.2857144 ,   7.        ,   8.428572  ,  34.57143   ,
        14.571428  ,   9.571428  ,  18.285715  ,  15.571428  ,
        16.571428  ,   6.285714  ,   3.4285715 ,   5.714286  ,
        40.714287  ,  14.285714  ,  12.857142  ,  31.857143  ,
        60.714287  ,   6.571429  ,   0.2857143 ,   4.285714  ,
       140.42857   ,  27.        ,   6.714286  ,   8.571428  ,
         9.        ,   2.142857  ,   3.        ,  69.        ,
        62.42857   ,  18.571428  ,  70.        ,  90.14286   ,
        37.57143   ,   9.        ,   6.        ,  33.57143   ,
         2.2857144 ,  37.        , 106.42857   ,  46.        ,
        12.285714  ,   3.142857  ,   0.2857143 ,   0.2

In [44]:
def generate_histogram(descriptors, centers): # descriptors is for a single image
    hist_arr = []
    num_bins = len(centers)
    for row in descriptors:
        min_dist = 9999999999999999999
        min_center_idx = -1
        center_idx = -1
        for center in centers:
            center_idx = center_idx + 1
            dist = np.linalg.norm(row - center)
            if dist < min_dist:
                min_dist = dist
                min_center_idx = center_idx
        hist_arr.append(min_center_idx)
    hist = np.histogram(hist_arr, bins=num_bins)
#     plt.hist(hist_arr, bins=num_bins)
#     plt.show()
    return hist

In [45]:
train_x_hists = []
for (image, label, kp, descriptors) in train_x:
    train_x_hists.append(generate_histogram(descriptors, centers)[0])

test_x_hists = []
for (image, label, kp, descriptors) in test_x:
    test_x_hists.append(generate_histogram(descriptors, centers)[0])

In [46]:
# use this later once we have our test and train cleaned up


model = OneVsRestClassifier(SVC()) # kernel='linear'
model.fit(train_x_hists, train_y)
prediction = model.predict(test_x_hists)
print(sk.metrics.classification_report(test_y, prediction))



# svclassifier = sk.svm.SVC(kernel='linear')
# svclassifier.fit(x_train, y_train)
# y_test_pred = svclassifier.predict(x_test)

              precision    recall  f1-score   support

           0       0.36      0.57      0.44         7
           1       1.00      0.29      0.44         7
           2       0.38      0.43      0.40         7

   micro avg       0.43      0.43      0.43        21
   macro avg       0.58      0.43      0.43        21
weighted avg       0.58      0.43      0.43        21



/home/andrew/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/andrew/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/andrew/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
